In [1]:
import pandas as pd
import ipywidgets as widgets

pricing_path = "savingsplanrates.csv"
usage_path = "currfile.csv"

In [2]:
saving_plans = pd.read_csv(pricing_path).drop(
    ["purchaseoption", "plantype", "sku", "ratecode", "unit", "contractlength", "rate_type", "discountedsku"], axis=1
)
saving_plans = saving_plans.query(
    "servicecode=='AmazonEC2' and operation in ['RunInstances','RunInstances:0002', 'Hourly']"
).drop("servicecode", axis=1)
saving_plans

,usagetype,operation,currency,rate,od_rate,unit_savings,perc_savings
10,EUC1-BoxUsage:c5.xlarge,RunInstances:0002,USD,0.2950,0.3780,0.0830,0.219577
16,BoxUsage:c5.xlarge,RunInstances,USD,0.0820,0.1700,0.0880,0.517647
20,BoxUsage:m5.xlarge,RunInstances,USD,0.0970,0.1920,0.0950,0.494792
25,BoxUsage:p3.2xlarge,RunInstances:0002,USD,2.1630,3.4280,1.2650,0.369020
26,BoxUsage:t2.micro,RunInstances:0002,USD,0.0104,0.0162,0.0058,0.358025
...,...,...,...,...,...,...,...
283,BoxUsage:m5a.large,RunInstances:0002,USD,0.1350,0.1780,0.0430,0.241573
285,BoxUsage:t3.large,RunInstances:0002,USD,0.0689,0.1108,0.0419,0.378159
293,USW1-BoxUsage:m5a.large,RunInstances:0002,USD,0.1510,0.1930,0.0420,0.217617
303,USW2-BoxUsage:r5a.large,RunInstances,USD,0.0590,0.1130,0.0540,0.477876


In [3]:
curr = pd.read_csv("currfile.csv", sep="|").drop(["lineitem/lineitemdescription"], axis=1)
curr.columns = [
    "operation",
    "usagetype",
    "timeinterval",
    "productcode",
    "usageamount",
    "ondemandrate",
]

DatetimePicker(value=None)

In [5]:
curr_ec2 = curr.query("operation in ['RunInstances','RunInstances:0002', 'Hourly']")
curr_ec2[["start_time", "end_time"]] = curr_ec2.timeinterval.str.split("/", expand=True)
curr_ec2["start_time"] = pd.to_datetime(curr_ec2.start_time, utc=True)
curr_ec2["end_time"] = pd.to_datetime(curr_ec2.end_time, utc=True)

C:\Temp\ipykernel_10404\2010502795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_ec2[["start_time", "end_time"]] = curr_ec2.timeinterval.str.split("/", expand=True)
C:\Temp\ipykernel_10404\2010502795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_ec2[["start_time", "end_time"]] = curr_ec2.timeinterval.str.split("/", expand=True)
C:\Temp\ipykernel_10404\2010502795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [6]:
saving_plans[["Region", "_", "instance_type"]] = (
    saving_plans.usagetype.str.split(r"-|:").apply(lambda x: [None] + x if len(x) == 2 else x).apply(pd.Series)
)
saving_plans

,usagetype,operation,currency,rate,od_rate,unit_savings,perc_savings,Region,_,instance_type
10,EUC1-BoxUsage:c5.xlarge,RunInstances:0002,USD,0.2950,0.3780,0.0830,0.219577,EUC1,BoxUsage,c5.xlarge
16,BoxUsage:c5.xlarge,RunInstances,USD,0.0820,0.1700,0.0880,0.517647,None,BoxUsage,c5.xlarge
20,BoxUsage:m5.xlarge,RunInstances,USD,0.0970,0.1920,0.0950,0.494792,None,BoxUsage,m5.xlarge
25,BoxUsage:p3.2xlarge,RunInstances:0002,USD,2.1630,3.4280,1.2650,0.369020,None,BoxUsage,p3.2xlarge
26,BoxUsage:t2.micro,RunInstances:0002,USD,0.0104,0.0162,0.0058,0.358025,None,BoxUsage,t2.micro
...,...,...,...,...,...,...,...,...,...,...
283,BoxUsage:m5a.large,RunInstances:0002,USD,0.1350,0.1780,0.0430,0.241573,None,BoxUsage,m5a.large
285,BoxUsage:t3.large,RunInstances:0002,USD,0.0689,0.1108,0.0419,0.378159,None,BoxUsage,t3.large
293,USW1-BoxUsage:m5a.large,RunInstances:0002,USD,0.1510,0.1930,0.0420,0.217617,USW1,BoxUsage,m5a.large
303,USW2-BoxUsage:r5a.large,RunInstances,USD,0.0590,0.1130,0.0540,0.477876,USW2,BoxUsage,r5a.large


In [23]:
from IPython.display import display
import numpy as np

pd.options.display.max_rows = 2
default_start_date = pd.to_datetime("2001-11-14", utc=True)
default_end_date = pd.to_datetime("2022-11-20", utc=True)


dropdown_operation = widgets.Dropdown(options=["RunInstances", "RunInstances:0002", "Hourly"], description="operation")
dropdown_usage_type = widgets.Dropdown(options=saving_plans.usagetype.unique(), description="usage type")
commitment_amount = widgets.IntSlider(value=3, min=0, max=50, step=1, description="commitment")
start_date_picker = widgets.DatetimePicker(description="Start Date", value=default_start_date)
end_date_picker = widgets.DatetimePicker(description="End Date", value=default_end_date)


def render_choice(start_date_pck, end_date_pck, dropdown_operation, dropdown_usage_type, commitment_amount):
    chosen_plan = saving_plans.query("usagetype == @dropdown_usage_type and operation == @dropdown_operation")
    display(chosen_plan)
    discount_rate = chosen_plan.rate.values[0]
    commited_hours = commitment_amount / discount_rate
    print(
        f"Commited Machine Hours: {commited_hours:.1f}h",
    )
    od_rate = chosen_plan.od_rate.values[0]
    print(f"Equivalent on Demand Commited Price: {(commited_hours*od_rate):.1f}$")
    print(f"Commited Price for Discounted Hours : {(commited_hours*discount_rate):.1f}$")
    start_date = pd.to_datetime(start_date_pck, utc=True)
    end_date = pd.to_datetime(end_date_pck, utc=True)
    usage_df = curr_ec2.query(
        "start_time >= @start_date and end_time <= @end_date and usagetype == @dropdown_usage_type and operation == @dropdown_operation"
    )
    display(usage_df.sort_values("start_time"))
    used_hours = usage_df.usageamount.sum()
    print(f"Used Hours: {used_hours:.1f}h")
    print(f"Used Commited Budget: {np.min([used_hours,commited_hours])*100/commited_hours:.1f}%")

    missing_hours = np.max([used_hours - commited_hours, 0])
    # if missing_hours > 0:
    print(f"Extra On-Demand to pay: {od_rate*missing_hours:.1f}$ ({missing_hours:.1f} extra hours)")
    print("----------------------")
    print(f"Total to pay: {np.min([used_hours,commited_hours])*discount_rate + od_rate*missing_hours :.1f}$")
    print(f"effect: {np.min([used_hours,commited_hours])*discount_rate + od_rate*missing_hours :.1f}$")

    # historic_cropped = curr_and_savings.query("start_time >= @start_date and end_time <= @end_date").assign(_="_")


widgets.interact(
    render_choice,
    start_date_pck=start_date_picker,
    end_date_pck=end_date_picker,
    dropdown_operation=dropdown_operation,
    dropdown_usage_type=dropdown_usage_type,
    commitment_amount=commitment_amount,
)

interactive(children=(DatetimePicker(value=Timestamp('2001-11-14 00:00:00+0000', tz='UTC'), description='Start…

<function __main__.render_choice(start_date_pck, end_date_pck, dropdown_operation, dropdown_usage_type, commitment_amount)>

In [ ]:
default_start_date = pd.to_datetime("2022-11-14", utc=True)
default_end_date = pd.to_datetime("2022-11-20", utc=True)
hours = pd.date_range(start=default_start_date, end=default_end_date, freq="h")
hours = pd.DataFrame({"time": hours, "_": "_"})
historic_cropped = curr_and_savings.query("start_time >= @start_date and end_time <= @end_date").assign(_="_")
hours.merge(historic_cropped, on="_", how="left")

,time,_,operation,usagetype,timeinterval,productcode,usageamount,ondemandrate,discountedsku,currency,rate,od_rate,unit_savings,perc_savings,start_time,end_time
0,2022-11-14 00:00:00+00:00,_,RunInstances,EUC1-BoxUsage:r5a.4xlarge,2022-11-14T19:00:00Z/2022-11-14T20:00:00Z,AmazonEC2,1.000000,1.096,78CYHANWZQKDHDT6,USD,0.573,1.096,0.523,0.477190,2022-11-14 19:00:00+00:00,2022-11-14 20:00:00+00:00
1,2022-11-14 00:00:00+00:00,_,RunInstances,USW1-BoxUsage:r5a.4xlarge,2022-11-18T11:00:00Z/2022-11-18T12:00:00Z,AmazonEC2,1.000000,1.008,3QGUTWMS5CQT2QBR,USD,0.528,1.008,0.480,0.476190,2022-11-18 11:00:00+00:00,2022-11-18 12:00:00+00:00
2,2022-11-14 00:00:00+00:00,_,RunInstances,BoxUsage:m3.large,2022-11-15T17:00:00Z/2022-11-15T18:00:00Z,AmazonEC2,1.000000,0.133,YGU2QZY8VPP94FSR,USD,0.086,0.133,0.047,0.353383,2022-11-15 17:00:00+00:00,2022-11-15 18:00:00+00:00
3,2022-11-14 00:00:00+00:00,_,RunInstances,USW1-BoxUsage:r5a.4xlarge,2022-11-15T21:00:00Z/2022-11-15T22:00:00Z,AmazonEC2,1.000000,1.008,3QGUTWMS5CQT2QBR,USD,0.528,1.008,0.480,0.476190,2022-11-15 21:00:00+00:00,2022-11-15 22:00:00+00:00
4,2022-11-14 00:00:00+00:00,_,RunInstances,EUC1-BoxUsage:r5.xlarge,2022-11-17T22:00:00Z/2022-11-17T23:00:00Z,AmazonEC2,1.000000,0.304,WADKRMX4T7XH7GYV,USD,0.159,0.304,0.145,0.476974,2022-11-17 22:00:00+00:00,2022-11-17 23:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11989610,2022-11-20 00:00:00+00:00,_,RunInstances,EUC1-APN3-AWS-Out-Bytes,2022-11-17T02:00:00Z/2022-11-17T03:00:00Z,AmazonEC2,0.000006,0.020,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-17 02:00:00+00:00,2022-11-17 03:00:00+00:00
11989611,2022-11-20 00:00:00+00:00,_,RunInstances,USE1-APN1-AWS-Out-Bytes,2022-11-14T11:00:00Z/2022-11-14T12:00:00Z,AmazonEC2,0.000006,0.020,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-14 11:00:00+00:00,2022-11-14 12:00:00+00:00
11989612,2022-11-20 00:00:00+00:00,_,RunInstances,EUC1-DataTransfer-In-Bytes,2022-11-14T16:00:00Z/2022-11-14T17:00:00Z,AmazonEC2,0.000001,0.000,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-14 16:00:00+00:00,2022-11-14 17:00:00+00:00
11989613,2022-11-20 00:00:00+00:00,_,Hourly,EBSOptimized:c5.large,2022-11-18T22:00:00Z/2022-11-18T23:00:00Z,AmazonEC2,1.000000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-18 22:00:00+00:00,2022-11-18 23:00:00+00:00


In [ ]:
curr.operation.value_counts()

In [ ]:
curr.query("description.str.contains(' for ')").operation.value_counts()

In [ ]:
curr  # .description.str.split(r'per|for',expand=True)#[0].str.replace("$","").str.replace("USD","").astype(float)#.value_counts()